In [1]:
#Importing the libraries
import cv2 
from ultralytics import YOLO
import time
import requests

In [2]:
# Load YOLO11n model
model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
# Telegram Bot Details
BOT_TOKEN = "..."  # Replace with your Telegram bot token
CHAT_ID = "..."  # Replace with your Telegram chat ID

In [4]:
# Define class names (Replace these with actual names and IDs)
CLASS_NAMES = {
    0: "Cabbage",
    1: "Capsicum",
    2: "orange",
    3: "tomato",
    4:"watermelon"
}

In [5]:
CLASS_COLORS = {
    0: (255, 0, 0),   # Blue for Cabbage
    1: (0, 255, 255), # Yellow for Capsicum
    2: (0, 165, 255), # Orange for Orange
    3: (0, 255, 0),   # Green for Tomato
    4: (0, 0, 255)    # Red for Watermelon
}

In [6]:
#Telegram related functions for message delivery
def send_telegram_alert(message):
    """Send a Telegram message"""
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    response = requests.post(url, json=payload)
    print("✅ Telegram alert sent!", response.json())

In [7]:
# Load video
video_path = "Input videos/Capsicum-2.mp4"  # Change this to your video file
output_path = "Output videos/Capsicum-2.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get FPS
# Video writer to save output
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [8]:
# Track low count alerts for each category
#low_count_start_time = {class_id: None for class_id in CLASS_NAMES}  
#alert_sent = {class_id: False for class_id in CLASS_NAMES} 

In [9]:
# Dictionaries to track detection history
detection_start_time = {}  # Tracks how long a category has been detected
low_count_start_time = {}  # Tracks how long a category is below 5
alert_sent = {}  # Prevent multiple alerts

In [10]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Error: Frame could not be read.")
        send_telegram_alert("❌ Error: Frame could not be read.")
        break

    # Run YOLOv8 detection
    results = model(frame)

    # Dictionary to count detected categories
    category_counts = {}  

    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])  # Class index
            confidence = float(box.conf[0])  # Confidence score
            if confidence >= 0.3:  # Filter valid detections
                category_counts[class_id] = category_counts.get(class_id, 0) + 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = CLASS_NAMES.get(class_id, "Unknown")
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 255, 0), 2)

    # Display category counts on video
    y_offset = 75
    for class_id, count in category_counts.items():
        label = CLASS_NAMES.get(class_id, "Unknown")
        cv2.putText(frame, f"{label}: {count}", (20, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 0, 255), 2)
        y_offset += 30

    # Save frame to video
    out.write(frame)

    # Current time
    current_time = time.time()

    # Track detected categories
    for class_id, count in category_counts.items():
        # Step 1: If category detected for the first time, start tracking
        if class_id not in detection_start_time:
            detection_start_time[class_id] = current_time
            alert_sent[class_id] = False  # Reset alert flag
        
        # Step 2: If detected for 5+ seconds, it qualifies for low-count tracking
        elif current_time - detection_start_time[class_id] >= 5:
            if count < 5:
                if class_id not in low_count_start_time:
                    low_count_start_time[class_id] = current_time  # Start tracking low count
                elif current_time - low_count_start_time[class_id] >= 5 and not alert_sent[class_id]:
                    send_telegram_alert(f"⚠️ Low {CLASS_NAMES[class_id]} Alert! Only {count} detected for 5+ seconds.")
                    alert_sent[class_id] = True  # Mark alert as sent
            else:
                low_count_start_time.pop(class_id, None)  # Reset low count timer if count is back to normal
                alert_sent[class_id] = False  # Reset alert flag

    # Show frame (optional)
    cv2.imshow("Vegetable Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 3 Capsicums, 62.7ms
Speed: 4.6ms preprocess, 62.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Capsicums, 67.3ms
Speed: 3.0ms preprocess, 67.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Capsicums, 56.1ms
Speed: 2.0ms preprocess, 56.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Capsicums, 55.0ms
Speed: 1.5ms preprocess, 55.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Capsicums, 53.1ms
Speed: 2.0ms preprocess, 53.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Capsicums, 71.8ms
Speed: 2.0ms preprocess, 71.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Capsicums, 53.2ms
Speed: 1.6ms preprocess, 53.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Capsicums, 54.0ms
Speed: 2.0ms preprocess, 54.0ms inference, 1.0ms postprocess per 

In [11]:
print(f"✅ Processed video saved as {output_path}")

✅ Processed video saved as Output videos/Capsicum-2.mp4
